# 学习如何进行文本处理
> 对于训练集，自然语言处理过程包括：
> 
>     中文分词、过滤停用词、特征权重计算、特征词选取。
> 
> 而对于测试集：
> 
>     特征权重计算、特征词


## 使用的包 导入

In [3]:
import pandas as pd
import numpy as np
import re
import jieba
import jieba.analyse
import matplotlib.pyplot as plt
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


## 原始数据及基本信息

In [2]:
data = pd.read_csv('Data/完整数据_暴雨_交通.csv')
data.head()

,id,bid,user_id,用户昵称,微博正文,头条文章url,发布位置,艾特用户,话题,转发数,评论数,点赞数,发布时间,发布工具,微博图片url,微博视频url,retweet_id
0,4632157211069133,Kduyt4u85,7073634525,中国交通,#我家门口那条路#5月1日20时至2日20时，新疆天山西部、西藏中部等地部分地方有雨夹雪。西...,NaN,NaN,NaN,"我家门口那条路,国家综合立体交通网规划纲要,中国路网",1,3,6,2021-05-01 21:30,微博 weibo.com,['https://wx3.sinaimg.cn/large/007IIftrly1gq34...,NaN,NaN
1,4632138676964878,Kdu4ztdSK,6360489687,昌江交警,"#五一我在岗##畅行中国,交警同行#五一，大雨滂沱，而你无阻。你是烈日暴雨下的指示牌，是人民...",NaN,NaN,"海南交警,中国交通频道,公安部交通管理局","五一我在岗,畅行中国,交警同行",0,0,0,2021-05-01 20:16,新版微博 weibo.com,['https://wx2.sinaimg.cn/large/006WrXVlgy1gq36...,NaN,NaN
2,4632091960806995,KdsRe3nW3,2098013967,高州天气,【高州天气预报】今晚到明天白天，多云，有（雷）阵雨局部大雨，气温22-29℃，相对湿度65%...,NaN,NaN,NaN,NaN,0,0,0,2021-05-01 17:10,微博 weibo.com,NaN,NaN,NaN
3,4632061941647124,Kds4O6Uuw,5323286636,广德公安在线,#我在岗位上#【狂风暴雨中交警保畅通】4月30日晚，狂风暴雨突然袭来。@广德交警面对恶劣天气...,NaN,NaN,广德交警,"我在岗位上,我为群众办实事",0,0,1,2021-05-01 15:11,360安全浏览器,['https://wx1.sinaimg.cn/large/005OfY1Sly1gq2y...,NaN,NaN
4,4632029499493949,KdretDPOd,2578964252,广德交警,#我为群众办实事#五一前夕，暴雨来袭！广德公安交警在岗在位，加强队所联勤，全力疏导交通、清理...,NaN,NaN,"公安部交通管理局,安徽公安交警在线,广德公安在线,宣城公安交警在线",我为群众办实事,0,0,0,2021-05-01 13:02,荣耀20 PRO,['https://wx2.sinaimg.cn/large/99b7df1cly1gq2u...,NaN,NaN


In [3]:
data.shape

(15155, 17)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15155 entries, 0 to 15154
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          15155 non-null  int64  
 1   bid         15155 non-null  object 
 2   user_id     15155 non-null  int64  
 3   用户昵称        15155 non-null  object 
 4   微博正文        15155 non-null  object 
 5   头条文章url     21 non-null     object 
 6   发布位置        1127 non-null   object 
 7   艾特用户        4092 non-null   object 
 8   话题          10761 non-null  object 
 9   转发数         15155 non-null  int64  
 10  评论数         15155 non-null  int64  
 11  点赞数         15155 non-null  int64  
 12  发布时间        15155 non-null  object 
 13  发布工具        14913 non-null  object 
 14  微博图片url     6400 non-null   object 
 15  微博视频url     5130 non-null   object 
 16  retweet_id  0 non-null      float64
dtypes: float64(1), int64(5), object(11)
memory usage: 2.0+ MB


In [6]:
data.isnull().sum()

id                0
bid               0
user_id           0
用户昵称              0
微博正文              0
头条文章url       15134
发布位置          14028
艾特用户          11063
话题             4394
转发数               0
评论数               0
点赞数               0
发布时间              0
发布工具            242
微博图片url        8755
微博视频url       10025
retweet_id    15155
dtype: int64

## 随机的200行，测试代码

In [7]:
df = data.sample(n=200, replace=False, random_state=1)
# df.reset_index(drop=True, inplace=True)
df.shape

(200, 17)

In [8]:
df['微博正文']

6826     【面对暴雨，请收好这份#避险生存指南#】#蜀黍带你涨姿势#近日来，河南两小时暴雨记录又被刷新...
1432     #河南高速路况#截至2021年9月9日,19:00，目前省内高速通行情况：1、因收费站临时抢...
6965     郑州大暴雨灾害这波，今天回来又看到一交通事故，一个大叔满身血淋淋坐在车前，前几日杭州电瓶车骑...
12476    【暖心！#交警暴雨中执勤频频被路人送伞#】6月10日17时许，浙江温州泰顺县一路口严重拥堵，...
3023     【湖南多地迎暴雨！这些收费站仍在管制中】今日晚间，湖南部分地区突降暴雨，905中国交通广播派...
                               ...                        
3014     暴雨来袭，行人户外活动谨防雷电！注意安全！转发，提醒更多人↓↓↓#长沙打雷##交通安全不放假...
6469     #河南高速路况#截至2021年7月29日11:05，1、因积水，禁止上下站的收费站有：菏宝高...
13372    #大众观天下#【暖心！#交警暴雨中执勤频频被路人送伞#】6月10日17时许，浙江温州泰顺县一...
11309    珠海交通【突发：受暴雨影响，梅界路交蓝盾路路口暂时禁止通行！】受暴雨影响，梅界路交蓝盾路奥园...
5623     【#外卖小哥雷电暴雨中指挥交通#：能帮大家尽快回家挺开心】据@看看新闻KNEWS消息：近日，...
Name: 微博正文, Length: 200, dtype: object

### 1 文本去噪
去除一些无用的信息；特殊的符号

In [7]:
def clean_noise(text):
    text = text.replace(u'\xa0', u' ')      # 去除 \xa0     不间断空白符 
    text = text.replace(u'\u3000', u' ')    # 去除 \u3000   全角的空白符
    
    text = re.sub(r"(回复)?(//)?\s*@\S*?\s*(:| |$)", "", text)  # 去除正文中的@和回复/转发中的用户名
    text = re.sub(r"\[\S+\]", "", text)     # 去除表情符号
    # text = re.sub(r"#\S+#", "", text)     # 去除话题内容
    URL_REGEX = re.compile(
        r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))',
        re.IGNORECASE)
    text = re.sub(URL_REGEX, "", text)      # 去除网址
    
    EMAIL_REGEX = re.compile(r"[-a-z0-9_.]+@(?:[-a-z0-9]+\.)+[a-z]{2,6}", re.IGNORECASE)
    text = re.sub(EMAIL_REGEX, "", text)    # 去除邮件 
    
    text = text.replace("转发微博", "")      # 去除无意义的词语
    text = text.replace("网页链接", "")
    text = re.sub(r"\s+", " ", text)        # 合并正文中过多的空格

    text = re.sub(r"\d{2,4}年|\d{1,2}月|\d{1,2}日|\d{1,2}时|\d{1,2}分| \d{1,2}点", "", text) # 去除 日期 时间
    text = re.sub(r"\d", "", text)
    
    return text.strip()


测试  和改进 clean_noise() 函数

In [10]:
print(clean_noise("太过分了@Rexzhenghao //@Janie_Zhang:招行最近负面新闻越来越多呀...1234"))
print(clean_noise("希望你?得好?我本＂@Pete三姑父\xa0肥血?史＂[晕][哈哈]@Pete三姑父"))
print(clean_noise("回复@钱旭明QXM:[嘻嘻][嘻嘻] //@钱旭明QXM:杨大哥[good][good]"))
print(clean_noise("【#赵薇#：正筹备下一部电影 但不是青春片....http://t.cn/8FLopdQ"))
print(clean_noise("[酷]//@芊如_GZ:[哈哈]// @布丁clout : 哈哈 // @audrey-panda :试了我家那位，说我属于＂放心的没法看的＂[晕] // @芊如_GZ :[晕]"))

太过分了招行最近负面新闻越来越多呀...
希望你?得好?我本＂肥血?史＂
杨大哥
【#赵薇#：正筹备下一部电影 但不是青春片....
哈哈 试了我家那位，说我属于＂放心的没法看的＂


In [11]:
print(re.findall(r'[\u4e00-\u9fa5]+', '这,!是测试12用例')) #匹配汉字
print(re.sub("\d", "", '这,!是测试12用例'))
print(''.join(re.findall('[\u4e00-\u9fa5]', '这,!是112测$$试12用例')))

print(re.sub(r"(\d{2,4}年\d{1,2}月\d{1,2}日)","","测试时间是2021年05月30日16时47分")) # 去除日期，时间
print(re.sub(r"(\d{1,2}月\d{1,2}日)", "", "测试时间是2021年05月30日16时47分")) # 去除日期 
print(re.sub(r"(\d{1,2}时\d{1,2}分)","","测试时间是21年05月30日16时47分")) # 去除时间
print(re.sub("\d", "","测试时间是21年05月30日16时47分" ))
print(re.sub(r"\d{2,4}年|\d{1,2}月|\d{1,2}日|\d{1,2}时|\d{1,2}分","","测试时间是2021年30日16时47分"))

print(re.sub(r"\d", "", "测试时间是21年05月30日16时47分"))

['这', '是测试', '用例']
这,!是测试用例
这是测试用例
测试时间是16时47分
测试时间是2021年16时47分
测试时间是21年05月30日
测试时间是年月日时分
测试时间是
测试时间是年月日时分


In [14]:
txt ="发布了头条文章：《【交管前沿（34）】暴雨期间，救助出行群众于危急之中》@公安部交通管理局@河南交警@平安中原@平安鹤壁O【交管前沿（34）】暴雨期间，救助出行群众于危急之中"
# txt ="#实时路况信息#游仙区绵梓路魏城段现目前暴雨骤至，能见度低，请过往车辆保持车距，注意行车安全。@中国交通频道·四川@平安绵阳@绵阳公安交警@微游仙L绵阳交警支队直属三大队的微博视频#压事故、整秩序、保平安#"
print(re.sub(r"@\S*?\s*(@|:| |$)","",txt))
print(re.sub(r"@\S*?\s*(:| |$)","",txt))
print(clean_noise(txt))

发布了头条文章：《【交管前沿（34）】暴雨期间，救助出行群众于危急之中》河南交警平安鹤壁O【交管前沿（34）】暴雨期间，救助出行群众于危急之中
发布了头条文章：《【交管前沿（34）】暴雨期间，救助出行群众于危急之中》
发布了头条文章：《【交管前沿（）】暴雨期间，救助出行群众于危急之中》


In [12]:
text_tmp = "希望你?得好?我本＂@Pete三姑父\xa0肥血?史＂[晕][哈哈]@Pete三姑父"
text_tmp


'希望你?得好?我本＂@Pete三姑父\xa0肥血?史＂[晕][哈哈]@Pete三姑父'

In [13]:
text_tmp.replace(u'\xa0', u' ')

'希望你?得好?我本＂@Pete三姑父 肥血?史＂[晕][哈哈]@Pete三姑父'

In [14]:
tweets = df.loc[:, ['微博正文']]
tmp = tweets.iloc[6, 0]
tmp

'【雨中，你们坚守的样子真帅！】6月17日早上8点，正值出行早高峰，一场突袭的暴雨倾注了这座城市，为确保雨天辖区道路安全畅通，避免因大雨造成的交通事故，@南充交警一大队\xa0全体民辅警，冒雨在辖区各个主要路段执勤，有的甚至来不及换上雨衣，淋着雨指挥车辆，疏导交通，全力保障恶劣天气下市民出行畅通，成为大雨中最美丽的风景线。@四川公安@四川交警@四川长安网L南充公安的微博视频'

In [15]:
res1 = clean_noise(tmp)
res1

'【雨中，你们坚守的样子真帅！】早上点，正值出行早高峰，一场突袭的暴雨倾注了这座城市，为确保雨天辖区道路安全畅通，避免因大雨造成的交通事故，全体民辅警，冒雨在辖区各个主要路段执勤，有的甚至来不及换上雨衣，淋着雨指挥车辆，疏导交通，全力保障恶劣天气下市民出行畅通，成为大雨中最美丽的风景线。'

### 2 中文分词

这里只是学习，分词和停用词过滤后面一起实现

In [18]:

jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持
strs=["我来到北京清华大学","乒乓球拍卖完了","中国科学技术大学"]
for str in strs:
    seg_list = jieba.cut(str, use_paddle=True) # 使用paddle模式
    print("Paddle Mode: " + '/'.join(list(seg_list)))

seg_list = jieba.cut("我来到北京清华大学【】", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学【】", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

Paddle enabled successfully......


Paddle Mode: 我/来到/北京清华大学
Paddle Mode: 乒乓球/拍卖/完/了
Paddle Mode: 中国科学技术大学
Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学/ 【】
Default Mode: 我/ 来到/ 北京/ 清华大学/ 【/ 】
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造


In [20]:
# jieba.lcut() 直接返回列表
seg_list = jieba.lcut("我来到北京清华大学【】", cut_all=True)
# print("Full Mode: " + "/ ".join(seg_list))  # 全模式
print(seg_list)

['我', '来到', '北京', '清华', '清华大学', '华大', '大学', '【】']


In [21]:
# 载入词典
# 用法： jieba.load_userdict(file_name) # file_name 为文件类对象或自定义词典的路径
# 词典格式和 dict.txt 一样，一个词占一行；每一行分三部分：词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒。file_name 若为路径或二进制方式打开的文件，则文件必须为 UTF-8 编码。
# 词频省略时使用自动计算的能保证分出该词的词频。

In [22]:
text = '疯狂囤金，中国大妈震惊华尔街'
jieba.load_userdict('CiDian.txt')
wordlist = jieba.lcut(text)
wordlist

['疯狂', '囤金', '，', '中国大妈', '震惊', '华尔街']

### 3 过滤停用词

In [23]:
# 读取停用词列表
def get_stopword_list(file):
    with open(file, 'r', encoding='utf-8') as f:    # 
        stopword_list = [word.strip('\n') for word in f.readlines()]
    return stopword_list


In [24]:
# 分词 然后清除停用词语
def clean_stopword(str, stopword_list):
    result = []
    word_list = jieba.cut(str)   # 分词后返回一个列表  jieba.cut(）   返回的是一个迭代器
    for w in word_list:
        if w not in stopword_list:
            result.append(w)
   
    return result


python 判断字符串的内容是不是数字

In [25]:
def is_number(s):
    try:  # 如果能运行float(s)语句，返回True（字符串s是浮点数）
        float(s)
        return True
    except ValueError:  # ValueError为Python的一种标准异常，表示"传入无效的参数"
        pass  # 如果引发了ValueError这种异常，不做任何事情（pass：不做任何事情，一般用做占位语句）
    try:
        import unicodedata  # 处理ASCii码的包
        unicodedata.numeric(s)  # 把一个表示数字的字符串转换为浮点数返回的函数
        return True
    except (TypeError, ValueError):
        pass
    return False

In [26]:
str1 = '1'
print(is_number(str1))
str2 = '1.1'
print(is_number(str2))
str3 = '-1'
print(is_number(str3))
str4 = '我们'
print(is_number(str4))

True
True
True
False


In [27]:
stopword_list = get_stopword_list('stopwords/hit_stopwords.txt')

res23 = clean_stopword(res1, stopword_list)
res23= [w for w in res23]
print(res23)

['雨中', '坚守', '样子', '真帅', '早上', '点', '正值', '出行', '早', '高峰', '一场', '突袭', '暴雨', '倾注', '这座', '城市', '确保', '雨天', '辖区', '道路', '安全', '畅通', '避免', '大雨', '造成', '交通事故', '全体', '民辅警', '冒雨', '辖区', '主要', '路段', '执勤', '来不及', '换上', '雨衣', '淋着', '雨', '指挥', '车辆', '疏导', '交通', '全力', '保障', '恶劣', '天气', '下', '市民', '出行', '畅通', '成为', '大雨', '中', '最', '美丽', '风景线']


In [28]:
sum(list(map(is_number, res23)))

0

### 4.词性标注 
没有这个需求

### 5.文本去重 

统一处理

In [210]:
import difflib
 
def string_similar(s1, s2):
    return difflib.SequenceMatcher(None, s1, s2).quick_ratio()


print(string_similar('爱尔眼科沪滨医院', '沪滨爱尔眼科医院'))
print(string_similar('安定区妇幼保健站', '定西市安定区妇幼保健站'))
print(string_similar('广州市医院', '广东省中医院'))
print(string_similar("广州市医院 - 我和你","你和我 - 广州市医院"))

1.0
0.8421052631578947
0.5454545454545454
1.0


### 6.文本标记

手动标记

### 7.特征词选择

统一处理和学习，这个就比较麻烦了

In [219]:
content = "".join(res23) #
topK = 10
tags = jieba.analyse.extract_tags(content, topK=topK, withWeight=True)
tags

[('大雨', 0.3348499425784),
 ('畅通', 0.3313562621724),
 ('辖区', 0.32669345500319996),
 ('出行', 0.3154784998372),
 ('真帅', 0.23909535005799998),
 ('民辅警', 0.23909535005799998),
 ('淋着', 0.23909535005799998),
 ('雨中', 0.208698834984),
 ('风景线', 0.20579519478600003),
 ('冒雨', 0.20144072511)]

英文特征提取 sklearn

In [17]:
corpus = [
    "I have a dog.",
    "You have a dog and a cat.",
    "He books a book.",
    "No cost too great.",
]

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

counter = CountVectorizer() # 先使用默认参数
counter.fit(corpus)
X = counter.transform(corpus)

df=pd.DataFrame(X.toarray(),columns=counter.get_feature_names_out())
print(df)

   and  book  books  cat  cost  dog  great  have  he  no  too  you
0    0     0      0    0     0    1      0     1   0   0    0    0
1    1     0      0    1     0    1      0     1   0   0    0    1
2    0     1      1    0     0    0      0     0   1   0    0    0
3    0     0      0    0     1    0      1     0   0   1    1    0


In [19]:
print(counter.vocabulary_)

{'have': 7, 'dog': 5, 'you': 11, 'and': 0, 'cat': 3, 'he': 8, 'books': 2, 'book': 1, 'no': 9, 'cost': 4, 'too': 10, 'great': 6}


In [20]:
print(X.todense()) # X是一个稀疏矩阵，输出稠密化表示

[[0 0 0 0 0 1 0 1 0 0 0 0]
 [1 0 0 1 0 1 0 1 0 0 0 1]
 [0 1 1 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 1 0 0 1 1 0]]


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer() # 先使用默认参数
tfidf.fit(corpus)
X = tfidf.transform(corpus)

df=pd.DataFrame(X.toarray(),columns=tfidf.get_feature_names_out())
print(df)

        and     book    books       cat  cost       dog  great      have  \
0  0.000000  0.00000  0.00000  0.000000   0.0  0.707107    0.0  0.707107   
1  0.485461  0.00000  0.00000  0.485461   0.0  0.382743    0.0  0.382743   
2  0.000000  0.57735  0.57735  0.000000   0.0  0.000000    0.0  0.000000   
3  0.000000  0.00000  0.00000  0.000000   0.5  0.000000    0.5  0.000000   

        he   no  too       you  
0  0.00000  0.0  0.0  0.000000  
1  0.00000  0.0  0.0  0.485461  
2  0.57735  0.0  0.0  0.000000  
3  0.00000  0.5  0.5  0.000000  


In [22]:
print(tfidf.vocabulary_)
print(tfidf.get_feature_names_out())


{'have': 7, 'dog': 5, 'you': 11, 'and': 0, 'cat': 3, 'he': 8, 'books': 2, 'book': 1, 'no': 9, 'cost': 4, 'too': 10, 'great': 6}
['and' 'book' 'books' 'cat' 'cost' 'dog' 'great' 'have' 'he' 'no' 'too'
 'you']


In [267]:
print(X.todense())

[[0.         0.         0.         0.         0.         0.70710678
  0.         0.70710678 0.         0.         0.         0.        ]
 [0.48546061 0.         0.         0.48546061 0.         0.38274272
  0.         0.38274272 0.         0.         0.         0.48546061]
 [0.         0.57735027 0.57735027 0.         0.         0.
  0.         0.         0.57735027 0.         0.         0.        ]
 [0.         0.         0.         0.         0.5        0.
  0.5        0.         0.         0.5        0.5        0.        ]]


中文特征提取 sklearn + jieba

In [116]:
#数据
data=["移动共享，共享汽车，共享经济，共享单车",
     "财经栏目，财经政策，经济政策，共享经济"] 

# 分词
cut_data=[]
for s in data:
    cut_s = jieba.cut(s)
    l_cut_s=' '.join(list(cut_s))    
    cut_data.append(l_cut_s)
    print(l_cut_s)

print(cut_data)

# TF-IDF
transfer = TfidfVectorizer() #实例化一个转换器类
data_new = transfer.fit_transform(cut_data) #调用fit_transform()
print(data_new)
print(data_new)
print(transfer.get_feature_names_out())
print(data_new.toarray()) 
#构建成一个二维表：
df=pd.DataFrame(data_new.toarray(),columns=transfer.get_feature_names_out())
print(df)

移动 共享 ， 共享 汽车 ， 共享 经济 ， 共享 单车
财经 栏目 ， 财经 政策 ， 经济 政策 ， 共享 经济
['移动 共享 ， 共享 汽车 ， 共享 经济 ， 共享 单车', '财经 栏目 ， 财经 政策 ， 经济 政策 ， 共享 经济']
  (0, 1)	0.2935323404273021
  (0, 6)	0.20885067778197156
  (0, 4)	0.2935323404273021
  (0, 0)	0.8354027111278862
  (0, 5)	0.2935323404273021
  (1, 2)	0.5889689090267317
  (1, 3)	0.29448445451336586
  (1, 7)	0.5889689090267317
  (1, 6)	0.41905622959186595
  (1, 0)	0.20952811479593297
  (0, 1)	0.2935323404273021
  (0, 6)	0.20885067778197156
  (0, 4)	0.2935323404273021
  (0, 0)	0.8354027111278862
  (0, 5)	0.2935323404273021
  (1, 2)	0.5889689090267317
  (1, 3)	0.29448445451336586
  (1, 7)	0.5889689090267317
  (1, 6)	0.41905622959186595
  (1, 0)	0.20952811479593297
['共享' '单车' '政策' '栏目' '汽车' '移动' '经济' '财经']
[[0.83540271 0.29353234 0.         0.         0.29353234 0.29353234
  0.20885068 0.        ]
 [0.20952811 0.         0.58896891 0.29448445 0.         0.
  0.41905623 0.58896891]]
         共享        单车        政策        栏目        汽车        移动        经济  \
0  0.8354

In [118]:
data_new.data 

array([0.29353234, 0.20885068, 0.29353234, 0.83540271, 0.29353234,
       0.58896891, 0.29448445, 0.58896891, 0.41905623, 0.20952811])

In [271]:
from  sklearn.feature_extraction.text import CountVectorizer

data=[u'今年国庆节打算去海南岛度假',"享受生活，顺其自然。这就是生活!"]

#分词
cut_data=[]
for s in data:
    cut_s=jieba.cut(s)    
    l_cut_s=' '.join(list(cut_s))    
    cut_data.append(l_cut_s)

print(cut_data) 
#统计特征词出现次数
transfer = CountVectorizer(stop_words=["打算","就是"]) 
#实例化一个转换器类,
# # stop_words=["打算","就是"],去除不想要的词
data_new = transfer.fit_transform(cut_data)  #调用fit_transform()
print(data_new)
print(transfer.get_feature_names_out())
print(data_new.toarray()) 
#构建成一个二维表：
data=pd.DataFrame(data_new.toarray(),columns=transfer.get_feature_names_out())
print(data)

['今年 国庆节 打算 去 海南岛 度假', '享受 生活 ， 顺其自然 。 这 就是 生活 !']
  (0, 1)	1
  (0, 2)	1
  (0, 4)	1
  (0, 3)	1
  (1, 0)	1
  (1, 5)	2
  (1, 6)	1
['享受' '今年' '国庆节' '度假' '海南岛' '生活' '顺其自然']
[[0 1 1 1 1 0 0]
 [1 0 0 0 0 2 1]]
   享受  今年  国庆节  度假  海南岛  生活  顺其自然
0   0   1    1   1    1   0     0
1   1   0    0   0    0   2     1


# 全部流程整合 输出结果

### 使用的包 导入

In [2]:
import pandas as pd
import numpy as np
import re
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

### 1 文本去噪
去除一些无用的信息；特殊的符号

In [3]:
def clean_noise(text):
    text = text.replace(u'\xa0', u' ')      # 去除 \xa0     不间断空白符 
    text = text.replace(u'\u3000', u' ')    # 去除 \u3000   全角的空白符
    
    text = re.sub(r"(回复)?(//)?\s*@\S*?\s*(:| |$)", "", text)  # 去除正文中的@和回复/转发中的用户名
    text = re.sub(r"\[\S+\]", "", text)     # 去除表情符号
    # text = re.sub(r"#\S+#", "", text)     # 去除话题内容
    URL_REGEX = re.compile(
        r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))',
        re.IGNORECASE)
    text = re.sub(URL_REGEX, "", text)      # 去除网址
    
    EMAIL_REGEX = re.compile(r"[-a-z0-9_.]+@(?:[-a-z0-9]+\.)+[a-z]{2,6}", re.IGNORECASE)
    text = re.sub(EMAIL_REGEX, "", text)    # 去除邮件 
    
    text = text.replace("转发微博", "")      # 去除无意义的词语
    text = text.replace("网页链接", "")
    text = re.sub(r"\s+", " ", text)        # 合并正文中过多的空格

    text = re.sub(r"\d{2,4}年|\d{1,2}月|\d{1,2}日|\d{1,2}时|\d{1,2}分| \d{1,2}点", "", text) # 去除 日期 时间
    text = re.sub(r"\d", "", text)
    text = re.sub('[a-zA-Z]',"", text)
    
    return text.strip()


### 2/3 分词 过滤停用词

In [4]:
# 读取停用词列表
def get_stopword_list(file):
    with open(file, 'r', encoding='utf-8') as f:    # 
        stopword_list = [word.strip('\n') for word in f.readlines()]
    return stopword_list

# 分词 然后清除停用词语
def clean_stopword(str, stopword_list):
    result = []
    word_list = jieba.cut(str)   # 分词后返回一个列表  jieba.cut(）   返回的是一个迭代器
    for w in word_list:
        if w not in stopword_list:
            result.append(w)
   
    return result


### 7.特征词选择

统一处理和学习，这个就比较麻烦了

In [24]:
def word2vec(lsls):
    ls_str=[]
    for s in lsls:
        strs = ' '.join(s) 
        ls_str.append(strs)
    
    # TF-IDF
    transfer = TfidfVectorizer() #实例化一个转换器类
    data_new = transfer.fit_transform(ls_str) #调用fit_transform()
    #构建成一个二维表：
    df=pd.DataFrame(data_new.toarray(), columns=transfer.get_feature_names_out())
    
    withWeight = transfer.vocabulary_
    return df, withWeight

### 原始数据及基本信息, 随机的500行，测试代码

In [25]:
data = pd.read_csv('Data/完整数据_暴雨_交通.csv')
df = data.sample(n=50, replace=False, random_state=1)
df.reset_index(drop=True, inplace=True)
tweets = df.loc[:, ['微博正文']]
tweets.shape

(50, 1)

In [26]:
tweets['clean_word'] = tweets['微博正文'].apply(clean_noise)

In [27]:
stopword = get_stopword_list('stopwords/hit_stopwords.txt')
tweets['clean_stopwords'] = tweets['clean_word'].apply(clean_stopword, stopword_list=stopword)

In [28]:
lsls = tweets['clean_stopwords']
df, withWeight = word2vec(lsls)

df.head()

,一下,一分,一名,一场,一大,一定,一楼,一程,一线,一课,...,鱼塘,鹤壁,黄河路,黄色,黑车,黔西南州,默兹河,默默,鼓楼,鼓楼区
0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.14623,0.0,0.14623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
topK = 50
downK = int(len(withWeight)/3)

down_L = sorted(withWeight.items(), key=lambda item:item[1], reverse=False)
top_L = sorted(withWeight.items(), key=lambda item:item[1], reverse=True)

tags_del = list(map(lambda x: x[0], down_L[0:downK]))

tags_sel = list(map(lambda x: x[0], top_L[0:topK]))

df2 = df.drop(columns=tags_del, axis=1, inplace=False)
df2.shape

(50, 949)

In [34]:
print(tags_sel)

['鼓楼区', '鼓楼', '默默', '默兹河', '黔西南州', '黑车', '黄色', '黄河路', '鹤壁', '鱼塘', '魏城', '高铁', '高速公路', '高速', '高空槽', '高空作业', '高空', '高热量', '高温', '高栏', '高架', '高川', '高峰期', '高峰', '高处', '高位', '骤至', '骑着', '驾驶员', '驾驶', '驾车', '驻点', '驶离', '驱赶', '马路', '首报', '首义', '饱和', '食物', '飞仙', '风雨', '风险', '风景线', '风力', '频频', '频繁', '领导', '预防', '预计', '预警']


In [7]:
import pandas as pd
df = pd.read_excel('Data/完整数据_暴雨_交通_pre_tweets_TFIDF.xlsx', index_col=0)
df['Label'] = 1
df.to_excel('Data/完整数据_暴雨_交通_pre_tweets_TFIDF_Label.xlsx')

# 报告数据处理

In [6]:
line1 = "#积水路段提醒##暴雨中的河南公安力量#1、金桥宾馆门前地下隧道积水严重，无法通行；2、金水路南阳路向北300米路东塌陷大坑；3、黄河路郑纺机地下道积水严重，无法通行。@平安郑州@河南交警@河南交通广播@郑州交通广播"

print(f"line1 原始:\n{line1}")

line2 = clean_noise(line1)
print(f"line2 去噪:\n{line2}")

line3 =jieba.lcut(line2)
print(f"line3 分词:\n{line3}")

stopword = get_stopword_list('stopwords/hit_stopwords.txt')
# 清除停用词语
def clean_stopword(str, stopword_list):
    result = []
    word_list = jieba.cut(str)   # 分词后返回一个列表  jieba.cut(）   返回的是一个迭代器
    for w in word_list:
        if w not in stopword_list:
            result.append(w)
   
    return result

line4 =  clean_stopword(line2, stopword)
print(f"line4 去停用词:\n{line4}")

line1 原始:
#积水路段提醒##暴雨中的河南公安力量#1、金桥宾馆门前地下隧道积水严重，无法通行；2、金水路南阳路向北300米路东塌陷大坑；3、黄河路郑纺机地下道积水严重，无法通行。@平安郑州@河南交警@河南交通广播@郑州交通广播
line2 去噪:
#积水路段提醒##暴雨中的河南公安力量#、金桥宾馆门前地下隧道积水严重，无法通行；、金水路南阳路向北米路东塌陷大坑；、黄河路郑纺机地下道积水严重，无法通行。
line3 分词:
['#', '积水', '路段', '提醒', '##', '暴雨', '中', '的', '河南', '公安', '力量', '#', '、', '金桥', '宾馆', '门前', '地下隧道', '积水', '严重', '，', '无法', '通行', '；', '、', '金水路', '南阳', '路向', '北米', '路东', '塌陷', '大坑', '；', '、', '黄河路', '郑', '纺机', '地下道', '积水', '严重', '，', '无法', '通行', '。']
line4 去停用词:
['积水', '路段', '提醒', '##', '暴雨', '中', '河南', '公安', '力量', '金桥', '宾馆', '门前', '地下隧道', '积水', '严重', '无法', '通行', '金水路', '南阳', '路向', '北米', '路东', '塌陷', '大坑', '黄河路', '郑', '纺机', '地下道', '积水', '严重', '无法', '通行']
